In [ ]:
import pandas as pd  
import numpy as np   
import urllib       
import re           
import datetime     
import calendar     
import time         
import scipy         
from sklearn.cluster import KMeans 
from haversine import haversine 
import math          
import seaborn as sns 
import matplotlib.pyplot as plt
import os    
import nltk
from nltk.corpus import stopwords
import string
import xgboost as xgb
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import ensemble, metrics, model_selection, naive_bayes
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

## ***Vấn đề***

Dự đoán giá sản phẩm
- Input: Dữ liệu bao gồm: tên, tình trạng sản phẩm, nhãn hiệu,shipping, danh mục, ,mô tả.
- Output: giá của sản phẩm(price > 0)

## ***Giải nén, đọc dữ liệu***

In [ ]:
!apt-get install p7zip
!apt install --assume-yes p7zip-full
!7z x /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z -y
!7z x /kaggle/input/mercari-price-suggestion-challenge/test_stg2.tsv.zip -y

In [ ]:
train_df = pd.read_table("train.tsv")
print(train_df.shape)
train_df.head()

In [ ]:
test_df = pd.read_table("test_stg2.tsv")
print(test_df.shape)
test_df.head()

Một sản phẩm bao gồm các thuộc tính:
- name: tên của sản phẩm
- item_condition_id: tình trạng của sản phẩm(1 - 5)
- category_name: danh sách danh mục của sản phẩm
- brand_name: nhãn hiệu của sản phẩm
- shipping: 1 nếu người bán chịu phí vận chuyển, 0 ngược lại
- item_description: mô tả của sản phẩm

# ***Phân tích, đánh giá dữ liệu***


**1. Price**

In [ ]:
train_df.price.describe()

Giá trị trung bình của trainning set là 267, nhưng giá trị một số giá trị cự trị trên 100, giá trị lớn nhất là 2000, phân bố này số lệch nhiều về bên trái

In [ ]:
plt.subplot(1, 2, 1)
(train_df['price']).plot.hist(bins=50, figsize=(20,10), edgecolor='white',range=[0,250])
plt.xlabel('price+', fontsize=17)
plt.ylabel('frequency', fontsize=17)
plt.tick_params(labelsize=15)
plt.title('Price Distribution - Training Set', fontsize=17)

Đúng như dư đoán, phân bố lệch về bên trái. Ta sử dụng hàm đồng biến log() sẽ đưa đồ thị giá về gần dạng phân bố chuẩn.

In [ ]:
plt.subplot(1, 2, 2)
np.log(train_df['price']+1).plot.hist(bins=50, figsize=(20,10), edgecolor='white')
plt.xlabel('log(price+1)', fontsize=17)
plt.ylabel('frequency', fontsize=17)
plt.tick_params(labelsize=15)
plt.title('Log(Price) Distribution - Training Set', fontsize=17)
plt.show()

Phân bố đã có dạng phân bố chuẩn -> bài toán hồi quy

**2. Shipping**

In [ ]:
train_df.shipping.value_counts()/len(train_df)

In [ ]:
prc_shipBySeller = train_df.loc[train_df.shipping==1, 'price']
prc_shipByBuyer = train_df.loc[train_df.shipping==0, 'price']

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
ax.hist(np.log(prc_shipBySeller+1), color='#8CB4E1', alpha=1.0, bins=50,
       label='Price when Seller pays Shipping')
ax.hist(np.log(prc_shipByBuyer+1), color='#007D00', alpha=0.7, bins=50,
       label='Price when Buyer pays Shipping')
ax.set(title='Histogram Comparison', ylabel='% of Dataset in Bin')
plt.xlabel('log(price+1)', fontsize=17)
plt.ylabel('frequency', fontsize=17)
plt.title('Price Distribution by Shipping Type', fontsize=17)
plt.tick_params(labelsize=15)
plt.show()

Có 55% người mua phải chịu phí vận chuyển. Ngoài ra, mức giá trung bình mà người mua phải chịu vận chuyển thấp hơn mức giá trung bình mà người bán phải chịu phí. Điều này cũng hợp lý, do các sản phẩm cần có giá cả thấp đề bù vào phí vận chuyển.

**3. Category**

In [ ]:
print(train_df['category_name'].nunique())
print(test_df['category_name'].nunique())

Có tất cả 1287 giá trị category khác nhau trên tập huấn luyện.
Sau đây top 5 các giá trị category phổ biến: 

In [ ]:
train_df['category_name'].value_counts()[:5]

In [ ]:
test_df['category_name'].value_counts()[:5]

Top các danh sách danh mục trên tập huấn luyện và test tượng tự nhau, đều về woman, beauty chiếm số lượng lớn.

In [ ]:
train_df['category_name'].isnull().sum()

Như chúng ta thấy giá trị của items trong cột category tạo nên từ 3 category riêng(1 main, 2 sub). Bên cạnh đó cũng tồn tại 6327 items không có giá trị category. Phần ngay sau đây, sẽ xử lý, tách bỏ giá trị category_name của mỗi item thành các category riêng lẻ, và xử lý các item không nhãn category.

**4. Brand name**

In [ ]:
print(train_df['brand_name'].nunique())
print(train_df['brand_name'].isnull().sum())

Có 4809 giá trị khác nhau của brand_name trong trainning dataset. Vẫn có các sản phẩm không có nhãn -> sẽ được xử lý ở bước sau

In [ ]:
print("Train dataset: \n" + str(train_df['category_name'].value_counts()[:5]))
print("\nTest dataset: \n" + str(test_df['category_name'].value_counts()[:5]))

Giá trị brand_name trên tập train và test có tỷ lệ tương đồng.

Do chỉ có 4809 giá trị brand_name trên 1482535 giá trị dữ liệu và ở dạng text -> giải pháp: mã hóa sang dạng số -> giúp tối ưu bộ nhớ

**5. Item Condition**

In [ ]:
print("Train dataset:\n" + str(train_df['item_condition_id'].value_counts() / train_df.shape[0]))
print("\nTest dataset:\n" + str(test_df['item_condition_id'].value_counts() / test_df.shape[0]))

Các giá trị item_condition_id trên tập train và test có tỷ lệ tương đồng

**Kết luận**: 2 tập dữ liệu train và test có tỷ lệ tương đồng nhau, việc học trên tập train sẽ mang lại ý nghĩa khi cần dự đoán trên tập test.

#  ***Xử lý, mã hóa các trường dữ liệu***

**1. Category**: Như đã đề cập ở trên phần xử lý cột category, hàm giúp xử lý tách trường category thành 3 catogory đơn lẻ, và các items không có nhãn sẽ được ("Other","Other","Other").

In [ ]:
def cat_split(text):
    try: return text.split("/")
    except: return ("Other","Other","Other")

In [ ]:
train_df["cat_1"], train_df["cat_2"], train_df["cat_3"] = zip(*train_df["category_name"].apply(lambda cat: cat_split(cat)))

In [ ]:
test_df["cat_1"], test_df["cat_2"], test_df["cat_3"] = zip(*test_df["category_name"].apply(lambda cat: cat_split(cat)))

In [ ]:
print("Train \t Test\n")
print(str(train_df["cat_1"].nunique()) + " \t" + str(test_df["cat_1"].nunique()))
print(str(train_df["cat_2"].nunique()) +" \t" + str(test_df["cat_2"].nunique()))
print(str(train_df["cat_3"].nunique()) + " \t" + str(test_df["cat_3"].nunique()))

Sau khi tách ra, có tất cả 10 main category, và 113 subCatgory_1 và 879 subCategory_2. Số lượng trên tập cả 2 tương tự nhau, bước sau ta sẽ đi vào chi tiết từng category riêng biệt.

In [ ]:
import plotly.offline as py
import plotly.graph_objs as go

In [ ]:
def drawGraph(data, label, cnt=10):
    x = data.value_counts().index.values.astype('str')[:cnt]
    y = data.value_counts().values[:cnt]
    pct = [("%.2f"%(v*100))+"%"for v in (y/len(train_df))][:cnt]
    trace1 = go.Bar(x=x, y=y, text=pct)
    layout = dict(title= label,
              yaxis = dict(title='Count'),
              xaxis = dict(title='Category'))
    fig=dict(data=[trace1], layout=layout)
    py.iplot(fig)

In [ ]:
drawGraph(train_df['cat_1'], "Number of Items by Main Category in Train Dataset")

In [ ]:
drawGraph(test_df['cat_1'], "Number of Items by Main Category in Test Dataset")

Về loại các catogry trên 2 tập là tương tự nhau, Bên cạnh đó,top các loại danh mục dẫn đầu cũng tương tự: Women, Beauty (chiếm hơn 50%), theo sau là Kids, Electronics

In [ ]:
drawGraph(train_df['cat_2'], "Number of Items by Main Category in Train Dataset",15)

In [ ]:
drawGraph(test_df['cat_2'], "Number of Items by Main Category in Test Dataset",15)

Các loại danh mục con(subCategory2) và phân phối của chúng cũng tương tự nhau. Phân phối giữa các cat_2 khá đồng đều, không có giá trị vượt trội lên. 

- Do các nhãn các nhãn của các trường cat_1, cat_2, cat_3 là dạng text chiếm không gian lưu trữ, tốn bộ nhớ nên ta sẽ mã hóa các nhãn này thành dạng số bằng cách sử dụng LabeEncoder. 
- LabeEncoder là là một lớp tiện ích để giúp chuẩn hóa các nhãn sao cho chúng chỉ chứa các giá trị từ 0 đến n_classes-1. Nó cũng có thể được sử dụng để chuyển đổi các nhãn không phải số (miễn là chúng có thể băm và có thể so sánh được) thành các nhãn số.

In [ ]:
#le = LabelEncoder()

#le.fit(train_df["cat_1"].unique().tolist() + test_df["cat_1"].unique().tolist())
#train_df["cat_1"] = le.transform(train_df["cat_1"])

In [ ]:
#test_df["cat_1"] = le.transform(test_df["cat_1"])

In [ ]:
#le.fit(train_df["cat_2"].unique().tolist() + test_df["cat_2"].unique().tolist())
#train_df["cat_2"] = le.transform(train_df["cat_2"])

In [ ]:
#test_df["cat_2"] = le.transform(test_df["cat_2"])

In [ ]:
#le.fit(train_df["cat_3"].unique().tolist() + test_df["cat_3"].unique().tolist())
#train_df["cat_3"] = le.transform(train_df["cat_3"])

In [ ]:
#test_df["cat_3"] = le.transform(test_df["cat_3"])

In [ ]:
#train_df.head()

2. Brand name

Trước khi đi vào mã hóa, do vẫn còn tồn tại các có giá trị nan, ta sẽ thay thế giá trị nan -> Missing

In [ ]:
#train_df.brand_name.fillna(value="missing", inplace=True)

In [ ]:
#test_df.brand_name.fillna(value="missing", inplace=True)

Tương tự như catgory_name, ta sử mã hóa các nhãn của brand_name thành số bằng sử dụng LabelEncoder

In [ ]:
#le.fit(train_df["brand_name"].unique().tolist() + test_df["brand_name"].unique().tolist())

In [ ]:
#train_df["brand_name"] = le.transform(train_df["brand_name"])

In [ ]:
#test_df["brand_name"] = le.transform(test_df["brand_name"])

3. Item description

**Hàm xử lý text**

Mục đích của hàm làm bình thường hóa text, gồm 3 bước:
* ngắt text thành các câu, sau đó các câu thành các tokens(sử dụng sent_tokenize(), word_tokenize() của nltk)
* xóa dấu câu, các từ thuộc stop_word
* viết thường các tokens
* loại bỏ các tokens có chiều dài nhỏ hơn 3.

In [ ]:
stop = set(stopwords.words('english'))
def tokenize(text):
    regex = re.compile('[' +re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    text = regex.sub(" ", text)
        
    tokens_ = [word_tokenize(s) for s in sent_tokenize(text)]
    tokens = []
    for token_by_sent in tokens_:
        tokens += token_by_sent
    tokens = list(filter(lambda t: t.lower() not in stop, tokens))
    filtered_tokens = [w for w in tokens if re.search('[a-zA-Z]', w)]
    filtered_tokens = [w.lower() for w in filtered_tokens if len(w)>=3]
        
        return filtered_tokens

Trước khi mã hóa, do vẫn còn tồn tại các có giá trị nan, ta sẽ thay thế giá trị nan -> None

In [ ]:
#train_df.item_description.fillna(value="None", inplace=True)

In [ ]:
#test_df.item_description.fillna(value="None", inplace=True)

Thuộc tính item_description ở dạng text nên cần mã hóa số -> sử dụng Tf-idf
- Tf-idf viết tắt của Term Frequency–inverse Document Frequency. là 1 kĩ thuật sử dụng trong khai phá dữ liệu văn bản. Trọng số này được sử dụng để đánh giá tầm quan trọng của một từ trong một văn bản. Giá trị cao thể hiện độ quan trọng cao và nó phụ thuộc vào số lần từ xuất hiện trong văn bản nhưng bù lại bởi tần suất của từ đó trong tập dữ liệu. Một vài biến thể của tf-idf thường được sử dụng trong các hệ thống tìm kiếm như một công cụ chính để đánh giá và sắp xếp văn bản dựa vào truy vấn của người dùng.  Tf-idf cũng được sử dụng để lọc những từ stopwords trong các bài toán như tóm tắt văn bản và phân loại văn bản. Nó định lượng tầm quan trọng của từ cụ thể với từ vựng của bộ sưu tập tài liệu hoặc ngữ liệu. số liệu phụ 2 yếu tố: tần suất, nghịch đảo tần suất tài liệu. Với cách này sẽ giúp phạt nặng các từ phổ thông(a, the, and...), các từ xuất hiện nhiều và không đem lại bất ký thông tin gì.
- Vấn đề: kích thước transform cao nên cần giải số chiều lại 

In [ ]:
#tfidf_vec = TfidfVectorizer(min_df=10,
#                             max_features=180000,
#                            tokenizer=tokenize,
#                             ngram_range=(1, 2))
#full_tfidf = tfidf_vec.fit_transform(train_df["item_description"].values.tolist() + test_df["item_description"].values.tolist())

In [ ]:
#train_tfidf = tfidf_vec.transform(train_df['item_description'].values.tolist())

In [ ]:
#test_tfidf = tfidf_vec.transform(test_df['item_description'].values.tolist())

In [ ]:
#print(train_tfidf.shape)

- Với kích thước cao (306423) của ma trận tfidf của chúng ta, chúng ta cần giảm thứ nguyên của chúng bằng cách sử dụng kỹ thuật Singular Value Decomposition (SVD).
- Ta có thể giảm kích thước của mỗi vectơ từ thành n_components (30) bằng cách sử dụng SVD.

In [ ]:
#n_comp = 30
#svd_obj = TruncatedSVD(n_components=n_comp, algorithm="arpack")
#svd_obj.fit(full_tfidf)

In [ ]:
#train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))

In [ ]:
#test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))

Do các thuộc tính sau mã hóa, và giảm chiều chưa có nhãn, ta cần gán nhãn cho các thuộc tính đó.

In [ ]:
#train_svd.columns = ['svd_item_'+str(i) for i in range(n_comp)]
#test_svd.columns = ['svd_item_'+str(i) for i in range(n_comp)]

#train_df = pd.concat([train_df, train_svd], axis=1)
#test_df = pd.concat([test_df, test_svd], axis=1)

4. Name
    - Mã hóa tương tự như category: chúng ta cần loại bỏ các từng thông dụng -> không mang lại nhiều ý nghĩa, tác dụng cho train model

In [ ]:
#train_df.name.fillna(value="missing", inplace=True)

In [ ]:
#test_df.name.fillna(value="missing", inplace=True)

In [ ]:
#tfidf_vec = TfidfVectorizer(min_df=10,
#                             max_features=180000,
#                            tokenizer=tokenize,
#                             ngram_range=(1, 2))
#full_tfidf = tfidf_vec.fit_transform(train_df["name"].values.tolist() + test_df["name"].values.tolist())

In [ ]:
#train_tfidf = tfidf_vec.transform(train_df["name"].values.tolist())

In [ ]:
#test_tfidf = tfidf_vec.transform(test_df["name"].values.tolist())

In [ ]:
#n_comp = 30
#svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
#svd_obj.fit(full_tfidf)

In [ ]:
#train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))

In [ ]:
#test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))

In [ ]:
#train_svd.columns = ['svd_name_'+str(i) for i in range(n_comp)]
#test_svd.columns = ['svd_name_'+str(i) for i in range(n_comp)]
#train_df = pd.concat([train_df, train_svd], axis=1)
#test_df = pd.concat([test_df, test_svd], axis=1)

In [ ]:
train = pd.read_csv("/kaggle/input/database1/train_1/train_1.csv")
print(train.shape)
train.head()

In [ ]:
test = pd.read_csv("/kaggle/input/database1/test_1/test_1.csv")
print(test.shape)
test.head()

Đây là tập dữ liệu huấn luyện, test sau quá trình mã hóa gồm 70 trường thuộc tính, ngoài ra có thêm 1 thuộc tính price với tập huấn luyện.
Có các thuộc tính vẫn tồn tại trong tập huấn luyện và test mà không cần tới -> loại bỏ các thuộc tính đó ('test_id', "name", 'cat_1', 'cat_2', 'cat_3','train_id', 'category_name', 'price', 'item_description'). Các thuộc tính này đã được mã hóa và không cần sử dụng các thuộc tính gốc( ở dạng text).

# **Train**

Loại bỏ các thuộc tính không sử dụng khi train. 

In [ ]:
do_not_use_for_training = ['cat_1','test_id',"name",'cat_2','cat_3','train_id', 'category_name', 'price', 'item_description']
feature_names = [f for f in train.columns if f not in do_not_use_for_training]

Đổi giá tiền sang dạng log.

In [ ]:
y = np.log(train['price'].values + 1)

In [ ]:
from sklearn.model_selection import train_test_split
Xtr, Xv, ytr, yv = train_test_split(train[feature_names].values, y, test_size=0.2, random_state=0)

# *Modeling*

Chia tập dữ liệu huấn thành 2: huấn luyện, đánh giá: dùng để huấn luyện, đánh giá mô hình. Vì bài toán giờ đã trở thành dự đoán log() và hàm đánh giá của kaggle là Root Mean Squared Log Error(RMLSE).

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
def rmsle(Y, Y_pred):
    assert Y.shape == Y_pred.shape
    return np.sqrt(np.mean(np.square(Y_pred - Y )))

In [ ]:
def run_model(model, Xtr, ytr, Xv, yv):
    model.fit(Xtr, ytr)
    
    ypre = model.predict(Xv)
    
    mse = mean_squared_error(yv, ypre)
    r_sq = r2_score(yv, ypre)
    rmlse_1 = rmsle(yv, ypre)
    
    print("Mean Squared Error Value : "+"{:.2f}".format(mse))
    print("\nR-Squared Value : "+"{:.2f}".format(r_sq))
    print("\nRMLSE : "+"{:.2f}".format(rmlse_1))
    return model, mse, r_sq, rmlse_1

**XGBoost**

Ta chuyển các tập dữ liệu thành ma trận dữ liệu DMatrix, được sử dụng nội bộ trong XGBoost được tối ưu hóa cho cả hiệu quả bộ nhớ và tốc độ đào tạo

In [ ]:
dtrain = xgb.DMatrix(Xtr, label=ytr)

In [ ]:
dvalid = xgb.DMatrix(Xv, label=yv)

In [ ]:
dtest = xgb.DMatrix(test[feature_names].values)

In [ ]:
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

Trước khi chạy xgboost, ta cần cài đặt các thông số: thông số chung(General parameters), thông số tăng số(Booster parameters), thông số nhiệm vụ(Task parameters).
- Thông số chung: 
    - booster: sử dụng gbtree
    - nthread: Số luồng song song được sử dụng để chạy XGBoost
- Thông số tree booster: 
    - eat: là learning rate (0.9)    
    - max_depth: Độ sâu của cây, việc tăng giá trị này sẽ làm cho mô hình phức tạp hơn và có nhiều khả năng bị quá tải (15)
    - min_child_weight: Tổng trọng lượng cá thể tối thiểu (hessian) cần thiết ở child. Trong đây sử dụng hồi quy tuyến tính, tương ứng với số lượng cá thể tối thiểu cần có trong mỗi nút. (20)
    - lamda: trọng số, tăng giá trị này sẽ làm cho mô hình trở nên thận trọng hơn (2.0)
- Thông số nhiệm vụ
    - eval_metric: Các chỉ số đánh giá cho dữ liệu xác thực (rmlse)
    - objective: mục tiêu học tập ("reg":"squarederror": hồi quy với tổn thất bình phương)
    - n_estimators: Number of gradient boosted trees. Equivalent to number of boosting rounds.(200)

In [ ]:
xgb_par = {'min_child_weight': 50, 'eta': 0.05, 'colsample_bytree': 0.5, 'max_depth': 50,
            'subsample': 0.9, 'lambda': 2.0, 'nthread': -1, 'booster' : 'gbtree', 'silent': 1,
            'eval_metric': 'rmse', 'objective': 'reg:linear'}

model_1 = xgb.train(xgb_par, dtrain, 200, watchlist, early_stopping_rounds=20, maximize=False, verbose_eval=20)
print('Modeling RMSLE %.5f' % model_1.best_score)

**Ridge Regression**

In [ ]:
#from sklearn.linear_model import Ridge
#ridge_reg = Ridge()
#print("Ridge Regression")
#print("----------------")
#model_3, mse_3, r_sq_3, rmlse_3 = run_model(ridge_reg, Xtr, ytr, Xv, yv)

#Result:
 - Mean Squared Error Value : 0.46
 - R-Squared Value : 0.18
 - RMLSE : 0.68

**LGBM Regression**

In [ ]:
#import lightgbm
#lgbm_reg = lightgbm.LGBMRegressor()
#print("LGBM Regression")
#print("---------------")
#model_4, mse_4, r_sq_4, rmlse_4 = run_model(lgbm_reg, Xtr, ytr, Xv, yv)

#Result:
 - Mean Squared Error Value : 0.36
 - R-Squared Value : 0.35
 - RMLSE : 0.60

**Kết luận**

Các mô hình phổ biến, đơn giản đều cho kết quả thấp hơn so với xgboost. Đối với model xgboost, khi điều chỉnh giá trị của min_child_weight, max_depth, n_entimators (cụ thể tăng lên) thì rmsle giảm (0.6 -> 0.52) do thời gian kaggle không cho phép nên chỉ có thể điều chỉnh giá trị n_entimators lên 200. Lý do bài không được kết quả cao như các bài khác do XGBoost được đánh giá vô cùng mạnh mẽ khi có thể giải quyết các bài toán phi tuyến tính. Nhưng bên cạnh đó, đó là hạn chế của XGBoost giải quyết bài toán xử lý ngôn ngữ, Nhiệm vụ hồi quy liên quan đến việc dự đoán kết quả đầu ra liên tục ([trích nguồn trên Kaggle](https://www.kaggle.com/discussion/196542)). Nhiệm vụ của bài toán này lại chính điểm yếu của xgboost. Lý do nữa là cách xử lý text của bài khác các bài điểm cao -> score thấp hơn :v

**Submit**

In [ ]:
ytest = model_1.predict(dtest)

In [ ]:
submission = np.exp(ytest) - 1

In [ ]:
submission = pd.DataFrame(submission).reset_index()

In [ ]:
submission.columns=["test_id","price"]

In [ ]:
print(submission.shape)
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)